In [21]:
import os
import typing
from sklearn.gaussian_process.kernels import *
import numpy as np
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn.kernel_approximation import Nystroem
from sklearn.linear_model import BayesianRidge
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from lightgbm import LGBMClassifier, LGBMRegressor

from autograd import grad 
import tensorflow as tf
from keras.losses import categorical_crossentropy, mean_squared_error
from keras import layers
from keras.layers import *
from keras.metrics import AUC
from keras.callbacks import EarlyStopping
from keras.activations import *

import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import random
from PIL import Image
import logging
import time

from tqdm import tqdm

from flaml import AutoML

import eli5
from eli5.sklearn import PermutationImportance

import shap

In [22]:
# Cost function constants
COST_W_UNDERPREDICT = 25.0
COST_W_NORMAL = 1.0
COST_W_OVERPREDICT = 10.0

In [23]:
def cost_function(ground_truth: np.ndarray, predictions: np.ndarray) -> float:
    # Unweighted cost
    cost = (ground_truth - predictions) ** 2
    weights = np.ones_like(cost) * COST_W_NORMAL

    # Case i): underprediction
    mask_1 = predictions < ground_truth
    weights[mask_1] = COST_W_UNDERPREDICT

    # Case ii): significant overprediction
    mask_2 = (predictions >= 1.2*ground_truth)
    weights[mask_2] = COST_W_OVERPREDICT

    # Weigh the cost and return the average
    return np.mean(cost * weights)


In [24]:
#convert the cost function to a tensorflow loss function
def cost_function_tf(): 
  def cost_fn(y_true, y_pred):
    # Unweighted cost
    cost = (y_true - y_pred) ** 2
    weights = tf.ones_like(cost) * COST_W_NORMAL

    # Case i): underprediction
    mask_1 = tf.cast(y_pred < y_true, tf.float32)
    weights = weights + mask_1 * (COST_W_UNDERPREDICT - COST_W_NORMAL)

    # Case ii): significant overprediction
    mask_2 = tf.cast(y_pred >= 1.2*y_true, tf.float32)
    weights = weights + mask_2 * (COST_W_OVERPREDICT - COST_W_NORMAL)

    # Weigh the cost and return the average
    return tf.reduce_mean(cost * weights)
    
  return cost_fn


In [25]:
# Load the training dateset and test features
X = np.loadtxt('train_x.csv', delimiter=',', skiprows=1)
y = np.loadtxt('train_y.csv', delimiter=',', skiprows=1)

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=1)

ny = Nystroem(random_state=1, n_components=4000,kernel='rbf',gamma=900)
X_train = ny.fit_transform(X_train)
X_test = ny.transform(X_test)



In [26]:
model = tf.keras.Sequential([
    layers.Input(shape=(X_train.shape[1],)),
    # layers.Dense(2048, activation=relu),
    # layers.Dropout(0.1),
    # layers.Dense(2048, activation=relu),
    # layers.Dense(2048, activation=relu),
    # layers.Dense(2048, activation=relu),
    # layers.Dense(2048, activation=relu),
    layers.Dense(1, activation='linear')
])


In [32]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss=cost_function_tf(), metrics=[cost_function_tf()])
model.fit(X_train, y_train, epochs=1000, batch_size=128, validation_data=(X_test, y_test), verbose=1)

Epoch 1/1000
95/95 [==============================] - 1s 5ms/step - loss: 35.9084 - cost_fn: 35.9022 - val_loss: 59.6389 - val_cost_fn: 59.8009
Epoch 2/1000
95/95 [==============================] - 0s 4ms/step - loss: 36.0841 - cost_fn: 36.0761 - val_loss: 59.6197 - val_cost_fn: 59.7827
Epoch 3/1000
95/95 [==============================] - 0s 4ms/step - loss: 36.1386 - cost_fn: 36.1333 - val_loss: 59.4291 - val_cost_fn: 59.5954
Epoch 4/1000
95/95 [==============================] - 0s 4ms/step - loss: 35.9363 - cost_fn: 35.9338 - val_loss: 59.3985 - val_cost_fn: 59.5541
Epoch 5/1000
95/95 [==============================] - 0s 4ms/step - loss: 35.8856 - cost_fn: 35.9025 - val_loss: 59.0639 - val_cost_fn: 59.2295
Epoch 6/1000
95/95 [==============================] - 0s 4ms/step - loss: 36.0218 - cost_fn: 36.0230 - val_loss: 59.1393 - val_cost_fn: 59.3054
Epoch 7/1000
95/95 [==============================] - 0s 4ms/step - loss: 35.3451 - cost_fn: 35.3517 - val_loss: 59.2850 - val_cost_fn: 